# 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import wkt
from shapely.ops import unary_union
from shapely.geometry import box
from shapely.strtree import STRtree
from shapely.geometry.base import BaseGeometry
from shapely.geometry import Polygon, MultiPolygon, GeometryCollection
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, MultiPolygon
from matplotlib.patches import Polygon as MplPolygon
from matplotlib.collections import PatchCollection
import matplotlib.colors as mcolors
import matplotlib
from shapely.geometry import Point
from pyproj import Transformer
from shapely.ops import transform
from tqdm import tqdm
import ast
import folium
from shapely.affinity import translate
import requests
import re
from shapely.ops import nearest_points
from itertools import product
import os
from collections import defaultdict
import json
import time
from folium.features import GeoJsonTooltip
import branca.colormap as cm
from shapely.geometry import mapping as shapely_mapping
from bs4 import BeautifulSoup
from folium.plugins import MarkerCluster
from ast import literal_eval
from geopy.distance import geodesic
from shapely.geometry import shape
from folium import Choropleth, GeoJson
from folium.plugins import Fullscreen
import random
from math import radians, cos, sin, sqrt, atan2
from shapely.geometry import mapping
import glob
from haversine import haversine

# 행정동 경계 기반 격자데이터 생성하기

### geojson을 csv로 변환하기

In [19]:
# GeoJSON 파일 경로
geojson_path = './HangJeongDong_ver20241001.geojson'

# GeoJSON 파일 읽기
with open(geojson_path, 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# features에서 geometry와 properties 추출하여 데이터프레임으로 저장
records = []
for feature in geojson_data['features']:
    properties = feature['properties']
    geom = shape(feature['geometry']).wkt  # GeoJSON → Shapely Geometry → WKT 문자열
    properties['geometry'] = geom
    records.append(properties)

# 데이터프레임으로 저장
df = pd.DataFrame(records)

# CSV로 저장
df.to_csv('./서울행정동경계.csv', index=False, encoding='utf-8')

In [38]:
seoul_admin_boundary = pd.read_csv('./서울행정동경계.csv')

# 서울특별시 추출
seoul_admin_boundary = seoul_admin_boundary[seoul_admin_boundary['adm_nm'].str.contains('서울특별시')]

# 컬럼 추출 : 행정동명, 행정동코드, geometry
seoul_admin_boundary = seoul_admin_boundary[['adm_nm', 'adm_cd2', 'geometry']]

# 행정동명만 추출
seoul_admin_boundary['adm_nm'] = seoul_admin_boundary['adm_nm'].apply(lambda x: x.split()[-1])

# 컬럼명 변경
seoul_admin_boundary.rename(columns={
    'adm_nm': '행정동명',
    'adm_cd2': '행정동코드'
}, inplace=True)

# 행정동코드의 앞 8자리만 남기기
seoul_admin_boundary['행정동코드'] = seoul_admin_boundary['행정동코드'].astype(str).str[:8]

# 저장
seoul_admin_boundary.to_csv('./서울행정동경계_필요한것만추출.csv', index=False, encoding='utf-8')

### 실제 격자 생성

In [49]:
# 파일 로드
file_path = './서울행정동경계_필요한것만추출.csv'
seoul_admin_boundary = pd.read_csv(file_path)

# shapely객체 변환
seoul_admin_boundary['geometry'] = seoul_admin_boundary['geometry'].apply(wkt.loads)

# box 생성
total_bounds = unary_union(seoul_admin_boundary['geometry'].tolist()).bounds

minx, miny, maxx, maxy = total_bounds

# 위도, 경도 기준 근사치로 격자 넓이 설정
grid_width = 0.011
grid_height = 0.009

# 격자 생성
grid_polygons = []
x_coords = np.arange(minx, maxx, grid_width)
y_coords = np.arange(miny, maxy, grid_height)

for x in x_coords:
    for y in y_coords:
        grid_polygons.append(box(x, y, x + grid_width, y + grid_height))

# 격자를 데이터프레임에 저장
grid_wkt = [poly.wkt for poly in grid_polygons]
grid_df = pd.DataFrame({'geometry': grid_wkt})

grid_df.to_csv('./격자데이터.csv')

### 격자 오버레이 & 행정동명과 행정동코드 머지

In [52]:
# 파일 로드
grid_df = pd.read_csv('./격자데이터.csv')
admin_df = pd.read_csv('./서울행정동경계_필요한것만추출.csv')

# Shapely 객체로 변환
grid_df['geometry'] = grid_df['geometry'].apply(wkt.loads)
admin_df['geometry'] = admin_df['geometry'].apply(wkt.loads)

# 실제 오버레이 수행 및 병합
result_data = []

for _, grid_row in grid_df.iterrows():
    grid_geom = grid_row['geometry']
    for _, admin_row in admin_df.iterrows():
        admin_geom = admin_row['geometry']
        if grid_geom.intersects(admin_geom):
            intersected_geom = grid_geom.intersection(admin_geom)
            if intersected_geom.is_empty:
                continue

            # 행정동명과 행정동코드 지지
            merged_data = grid_row.to_dict()
            merged_data['geometry'] = intersected_geom.wkt
            merged_data['행정동명'] = admin_row['행정동명']
            merged_data['행정동코드'] = admin_row['행정동코드']

            result_data.append(merged_data)

final_overlay_df = pd.DataFrame(result_data)

,Unnamed: 0,geometry,행정동명,행정동코드
0,13,POLYGON ((126.77569057122192 37.55501070369193...,공항동,11500620
1,14,POLYGON ((126.77569057122192 37.55501070369193...,공항동,11500620
2,15,POLYGON ((126.77569057122192 37.56531793601277...,공항동,11500620
3,43,POLYGON ((126.78669057122191 37.54601070369193...,공항동,11500620
4,44,POLYGON ((126.77569057122192 37.55501070369193...,공항동,11500620


In [64]:
final_overlay_df.to_csv('./서울시격자데이터_오버레이완료.csv', index=False, encoding='utf-8')

### 시각화

In [ ]:
seoul_center = [37.5665, 126.9780]
map_seoul = folium.Map(location=seoul_center, zoom_start=11, tiles='cartodbpositron')

# 폴리움에 격자추가
for _, row in final_overlay_df.iterrows():
    polygon = wkt.loads(row['geometry'])
    geo_json = folium.GeoJson(data=polygon.__geo_interface__,
                              style_function=lambda x: {
                                  'fillColor': '#4287f5',
                                  'color': 'blue',
                                  'weight': 0.5,
                                  'fillOpacity': 0.3
                              })
    geo_json.add_to(map_seoul)

map_seoul.save('./seoul_grid_overlay_map.html')
map_seoul

### 그리드ID 생성 및 불필요한 컬럼 제거

In [76]:
df = pd.read_csv("./서울시격자데이터_오버레이완료.csv", encoding='utf-8')

# Unnamed 컬럼 제거
df = df.loc[:, ~df.columns.str.startswith('Unnamed: 0')]

# grid_id 생성
df['grid_id'] = [f"G_{str(i).zfill(4)}" for i in range(1, len(df) + 1)]

# grid_id를 맨 앞으로 이동
cols = ['grid_id'] + [col for col in df.columns if col != 'grid_id']
df = df[cols]

# 저장
df.to_csv("./서울시격자데이터_그리드ID포함.csv", index=False, encoding='utf-8')

# 지적편집도 매핑하기

In [22]:
# 파일 로드
grid_df = pd.read_csv('./서울시격자데이터_그리드ID포함.csv', encoding='utf-8')
landuse_df = pd.read_csv('./merge/seoul_landuse_wkt.csv', encoding='utf-8')

# WKT -> Geometry
grid_df['geometry'] = grid_df['geometry'].apply(wkt.loads)
landuse_df['geometry'] = landuse_df['WKT'].apply(wkt.loads)

# 지적편집도 상위정보 분류
def classify_landuse(name):
    if pd.isna(name):
        return '기타'
    if '주거' in name:
        return '주거지역'
    elif '상업' in name:
        return '상업지역'
    elif '공업' in name:
        return '공업지역'
    elif '녹지' in name or '자연' in name:
        return '녹지지역'
    elif '관리' in name:
        return '관리지역'
    else:
        return '기타'

landuse_df['지적편집도_상위정보'] = landuse_df['DGM_NM'].apply(classify_landuse)

# 비율 계산 포함하여 교차 영역 구하기
detailed_ratio_info = {}
general_ratio_info = {}

landuse_records = landuse_df[['geometry', 'DGM_NM', '지적편집도_상위정보']].to_records(index=False)

tqdm.pandas()
for idx, row in tqdm(grid_df.iterrows(), total=len(grid_df)):
    gid = row['grid_id']
    gpoly = row['geometry']
    garea = gpoly.area if gpoly.is_valid else 1.0

    detail_ratio = {}
    general_ratio = {}

    for lu_geom, lu_name, lu_class in landuse_records:
        if not gpoly.intersects(lu_geom):
            continue
        intersection = gpoly.intersection(lu_geom)
        inter_area = intersection.area

        if pd.notna(lu_name):
            detail_ratio[lu_name] = detail_ratio.get(lu_name, 0) + inter_area
        if pd.notna(lu_class):
            general_ratio[lu_class] = general_ratio.get(lu_class, 0) + inter_area

    # 비율 정규화 (격자 기준)
    detail_ratio_normalized = {k: v / garea for k, v in detail_ratio.items()} if detail_ratio else {'없음': 1.0}
    general_ratio_normalized = {k: v / garea for k, v in general_ratio.items()} if general_ratio else {'없음': 1.0}

    detailed_ratio_info[gid] = detail_ratio_normalized
    general_ratio_info[gid] = general_ratio_normalized

# DataFrame 변환
info_df = pd.DataFrame({
    'grid_id': detailed_ratio_info.keys(),
    '지적편집도_상세정보': detailed_ratio_info.values(),
    '지적편집도_상위정보': general_ratio_info.values()
})


# 원래 격자 데이터와 병합
merged_df = pd.merge(grid_df, info_df, on='grid_id', how='left')

# 저장
output_path = "./서울시격자_지적편집도정보추가.csv"
merged_df.to_csv(output_path, index=False, encoding='utf-8')

output_path

100%|██████████████████████████████████████████████████████████████████████████████| 2427/2427 [06:35<00:00,  6.14it/s]


'./서울시격자_지적편집도정보추가.csv'

In [24]:
merged_df['지적편집도_상위정보'].head(30)

0                          {'녹지지역': 0.9814886899711561}
1                          {'녹지지역': 0.9433763168010767}
2                          {'녹지지역': 0.8065271222406061}
3                          {'녹지지역': 0.3162062270434837}
4                          {'녹지지역': 0.9907152885343161}
5     {'기타': 1.3922549220710221e-05, '녹지지역': 0.99797...
6                           {'녹지지역': 0.984722278978895}
7                          {'녹지지역': 0.9693888447549819}
8                          {'녹지지역': 0.8613329225813875}
9                           {'녹지지역': 0.985765690581208}
10                         {'녹지지역': 0.9998566255325073}
11                                        {'녹지지역': 1.0}
12                                        {'녹지지역': 1.0}
13                                        {'녹지지역': 1.0}
14                         {'녹지지역': 0.9971175868487029}
15                         {'녹지지역': 0.9448591891697685}
16                         {'녹지지역': 0.9738103460742646}
17                          {'녹지지역': 0.500649184

### 시각화

In [28]:
df = pd.read_csv("./서울시격자_지적편집도정보추가.csv")

df['geometry'] = df['geometry'].apply(wkt.loads)

target_col = '지적편집도_상세정보'  # 또는 '지적편집도_상위정보'

categories = df[target_col].dropna().unique()
colors = {cat: f"#{random.randint(0, 0xFFFFFF):06x}" for cat in categories}

center = [37.5665, 126.9780]  # 서울 중심 좌표
m = folium.Map(location=center, zoom_start=11, tiles="cartodbpositron")
Fullscreen().add_to(m)

for _, row in df.iterrows():
    geom = row['geometry']  # shapely Polygon
    category = row[target_col]

    if pd.isna(category):
        continue

    gj = folium.GeoJson(
        data=geom.__geo_interface__,
        style_function=lambda feature, color=colors[category]: {
            'fillColor': color,
            'color': color,
            'weight': 0.5,
            'fillOpacity': 0.6,
        },
        tooltip=folium.Tooltip(f"{target_col}: {category}")
    )
    gj.add_to(m)

m.save(f"지적편집도_{target_col}_시각화.html")

# 서울형 공립 키즈카페 매핑

In [66]:
# 파일 로드
grid_df = pd.read_csv("./서울시격자_지적편집도정보추가.csv", encoding='utf-8')
kids_df = pd.read_csv("./merge/서울형_키즈카페_행정동코드_수정완료.csv", encoding='utf-8')

# shapely Polygon 객체로 변환
grid_df['geometry'] = grid_df['geometry'].apply(wkt.loads)

# 위경도 컬럼 생성
kids_df['lon'] = kids_df['x좌표값']
kids_df['lat'] = kids_df['y좌표값']

# 시설정보 컬럼 생성
kids_df['서울형공립키즈카페_정보'] = kids_df.apply(lambda row: {
    '시설명': row['시설명'],
    '시설주소': row['기본주소'],
    '위도': row['lat'],
    '경도': row['lon']
}, axis=1)

# Point 생성
kids_df['point'] = kids_df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# 7. 각 point가 포함된 grid polygon 찾기
def find_grid_index(pt):
    for idx, row in grid_df.iterrows():
        if row['geometry'].intersects(pt):
            return idx
    return None

kids_df['grid_index'] = kids_df['point'].apply(find_grid_index)

# 병합을 위한 컬럼 초기화
grid_df['서울형공립키즈카페_정보'] = [[] for _ in range(len(grid_df))]

# 병합 수행
for _, row in kids_df.dropna(subset=['grid_index']).iterrows():
    idx = int(row['grid_index'])
    grid_df.at[idx, '서울형공립키즈카페_정보'].append(row['서울형공립키즈카페_정보'])

# 저장
grid_df.to_csv("./서울격자_서울형공립키즈카페포함.csv", index=False)

# 인증형키즈카페 매핑

In [81]:
tqdm.pandas()

# 격자 데이터 로드 (공립 키즈카페 병합 전 버전)
grid_df = pd.read_csv("./서울격자_서울형공립키즈카페포함.csv", encoding='utf-8')
grid_df['geometry'] = grid_df['geometry'].apply(wkt.loads)

# 인증제 키즈카페 데이터 로드
certified_df = pd.read_csv("./merge/서울형 인증제 키즈카페_행정동_코드맵핑_최종.csv", encoding='cp949')

# 위도/경도 -> Point
certified_df['lon'] = certified_df['경도']
certified_df['lat'] = certified_df['위도']
certified_df['point'] = certified_df.progress_apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# intersects 방식으로 격자 찾기
def find_grid_index_intersects(pt):
    for idx, row in grid_df.iterrows():
        if row['geometry'].intersects(pt):
            return idx
    return None

certified_df['grid_index'] = certified_df['point'].progress_apply(find_grid_index_intersects)

# 병합용 정보 생성
certified_df['인증제키즈카페_정보'] = certified_df.apply(lambda row: {
    '시설명': row['시설명'],
    '도로명주소': row['도로명주소'],
    '위도': row['lat'],
    '경도': row['lon']
}, axis=1)

# 격자에 병합
grid_df['인증제키즈카페_정보'] = [[] for _ in range(len(grid_df))]

for _, row in certified_df.dropna(subset=['grid_index']).iterrows():
    idx = int(row['grid_index'])
    grid_df.at[idx, '인증제키즈카페_정보'].append(row['인증제키즈카페_정보'])

# 저장
grid_df.to_csv("./서울시격자_인증제키즈카페포함.csv", index=False, encoding='utf-8')

100%|██████████████████████████████████████████████████████████████████████████████████| 53/53 [00:05<00:00,  9.54it/s]


In [83]:
unmatched_certified = certified_df[certified_df['grid_index'].isna()]
unmatched_preview = unmatched_certified[['시설명', '도로명주소', '위도', '경도']].reset_index(drop=True)

# 격자별 몇 개 시설이 들어갔는지 확인
grid_count_distribution = certified_df['grid_index'].value_counts().value_counts().sort_index()

unmatched_preview, grid_count_distribution

(Empty DataFrame
 Columns: [시설명, 도로명주소, 위도, 경도]
 Index: [],
 count
 1    44
 2     3
 3     1
 Name: count, dtype: int64)

# 일반키즈카페 매핑

In [92]:
tqdm.pandas()

# 데이터 로드
grid_df = pd.read_csv("./서울시격자_인증제키즈카페포함.csv", encoding='utf-8')
grid_df['geometry'] = grid_df['geometry'].apply(wkt.loads)

# 중복 제거용 데이터 드드
certified_df = pd.read_csv("./merge/서울형 인증제 키즈카페_행정동_코드맵핑_최종.csv", encoding='cp949')

# 인증제 위경도 기반 좌표셋 생성
certified_coords = set(
    tuple(round(pt, 5) for pt in (row['경도'], row['위도']))
    for _, row in certified_df.iterrows()
)

# 일반 키즈카페 데이터 드드
general_df = pd.read_csv("./merge/일반키즈카페_행정동_코드맵핑_수정완료.csv")
general_df['lon'] = general_df['경도']
general_df['lat'] = general_df['위도']
general_df['point'] = general_df.progress_apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# 중복 거거
general_df['rounded_coord'] = general_df.apply(lambda row: (round(row['lon'], 5), round(row['lat'], 5)), axis=1)
general_filtered = general_df[~general_df['rounded_coord'].isin(certified_coords)].copy()

# intersects 기반 격자 매핑 함수
def find_grid_index_intersects(pt):
    for idx, row in grid_df.iterrows():
        if row['geometry'].intersects(pt):
            return idx
    return None

general_filtered['grid_index'] = general_filtered['point'].progress_apply(find_grid_index_intersects)

# 병합용 딕셔너리 구성
general_filtered['일반키즈카페_정보'] = general_filtered.apply(lambda row: {
    '시설명': row['키즈카페명'],
    '도로명주소': row['도로명주소'],
    '위도': row['lat'],
    '경도': row['lon']
}, axis=1)

# 격자에 병합
grid_df['일반키즈카페_정보'] = [[] for _ in range(len(grid_df))]

for _, row in general_filtered.dropna(subset=['grid_index']).iterrows():
    idx = int(row['grid_index'])
    grid_df.at[idx, '일반키즈카페_정보'].append(row['일반키즈카페_정보'])

# 저장
grid_df.to_csv("서울시격자_일반키즈카페포함.csv", index=False, encoding='utf-8')

100%|████████████████████████████████████████████████████████████████████████████████| 379/379 [00:42<00:00,  8.89it/s]


In [94]:
# 실제 병합된 격자 수 확인
non_empty_count_general = sum([1 for i in grid_df['일반키즈카페_정보'] if len(i) > 0])
non_empty_count_general

265

# 유치원-초등학교 데이터 매핑

In [99]:
tqdm.pandas()

# 데이터로드
grid_df = pd.read_csv("./서울시격자_일반키즈카페포함.csv", encoding='utf-8')
grid_df['geometry'] = grid_df['geometry'].apply(wkt.loads)

school_df = pd.read_csv("./merge/유치원_초등학교_행정동코드_매핑완료.csv")

# Point 생성
school_df['lon'] = school_df['경도']
school_df['lat'] = school_df['위도']
school_df['point'] = school_df.progress_apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# intersects 기반 격자 매핑 함수
def find_grid_index_intersects(pt):
    for idx, row in grid_df.iterrows():
        if row['geometry'].intersects(pt):
            return idx
    return None

school_df['grid_index'] = school_df['point'].progress_apply(find_grid_index_intersects)

# 병합용 정보 구성
school_df['유치원_초등학교_정보'] = school_df.apply(lambda row: {
    '시설명': row['시설명'],
    '시설유형': row['시설유형'],
    '도로명주소': row['도로명 주소'],
    '위도': row['lat'],
    '경도': row['lon']
}, axis=1)

# 병합 컬럼 초기화
grid_df['유치원_초등학교_정보'] = [[] for _ in range(len(grid_df))]

# 병합 수행
for _, row in school_df.dropna(subset=['grid_index']).iterrows():
    idx = int(row['grid_index'])
    grid_df.at[idx, '유치원_초등학교_정보'].append(row['유치원_초등학교_정보'])

# 병합된 격자 수 확인
non_empty_count_school = sum([1 for i in grid_df['유치원_초등학교_정보'] if len(i) > 0])
print(f"유치원·초등학교 정보가 포함된 격자 수: {non_empty_count_school}")

# 저장
grid_df.to_csv("./서울시격자_유치원초등학교포함.csv", index=False, encoding='utf-8')

100%|██████████████████████████████████████████████████████████████████████████████| 1678/1678 [03:18<00:00,  8.46it/s]


유치원·초등학교 정보가 포함된 격자 수: 810


In [101]:
total_school_count = len(school_df)
mapped_school_count = school_df['grid_index'].notna().sum()
unmapped_school_count = total_school_count - mapped_school_count

total_school_count, mapped_school_count, unmapped_school_count

(1678, 1676, 2)

# 어린이집 데이터 매핑

In [108]:
# 데이터 로드
childcare_df = pd.read_csv("./merge/서울시_어린이집_행정동코드_최종완료.csv")
school_df = pd.read_csv("./merge/유치원_초등학교_행정동코드_매핑완료.csv")
grid_df = pd.read_csv("./서울시격자_유치원초등학교포함.csv", encoding='utf-8')

grid_df['geometry'] = grid_df['geometry'].apply(wkt.loads)

# 어린이집 데이터 전처리
childcare_df['lon'] = childcare_df['시설 경도(좌표값)_수정']
childcare_df['lat'] = childcare_df['시설 위도(좌표값)_수정']
childcare_df['point'] = childcare_df.progress_apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# 유치원·초등학교 데이터 중 "유치원" 또는 "어린이집" 유형의 좌표 추출
duplicate_coords = set(
    tuple(round(pt, 5) for pt in (row['경도'], row['위도']))
    for _, row in school_df.iterrows()
    if row['시설유형'] in ['어린이집', '유치원']
)

# 중복 제거된 어린이집만 필터링
childcare_df['rounded_coord'] = childcare_df.apply(lambda row: (round(row['lon'], 5), round(row['lat'], 5)), axis=1)
childcare_filtered = childcare_df[~childcare_df['rounded_coord'].isin(duplicate_coords)].copy()

# intersects 방식으로 격자 매핑
childcare_filtered['grid_index'] = childcare_filtered['point'].progress_apply(find_grid_index_intersects)

# 병합용 딕셔너리 구성
childcare_filtered['어린이집_정보'] = childcare_filtered.apply(lambda row: {
    '어린이집명': row['어린이집명'],
    '시설주소': row['상세주소'],
    '위도': row['lat'],
    '경도': row['lon']
}, axis=1)

# 병합 전 컬럼 초기화
grid_df['어린이집_정보'] = [[] for _ in range(len(grid_df))]

# 병합 수행
for _, row in childcare_filtered.dropna(subset=['grid_index']).iterrows():
    idx = int(row['grid_index'])
    grid_df.at[idx, '어린이집_정보'].append(row['어린이집_정보'])

# 실제 병합된 격자 수 확인
non_empty_count_childcare = sum([1 for i in grid_df['어린이집_정보'] if len(i) > 0])
non_empty_count_childcare

100%|██████████████████████████████████████████████████████████████████████████████| 3823/3823 [22:16<00:00,  2.86it/s]


1070

In [111]:
grid_df.to_csv("서울시격자_어린이집포함.csv", index=False, encoding='utf-8')

# 지역아동센터 데이터 매핑

In [115]:
tqdm.pandas()

# 데이터 로드
grid_df = pd.read_csv("./서울시격자_어린이집포함.csv", encoding='utf-8')
grid_df['geometry'] = grid_df['geometry'].apply(wkt.loads)

center_df = pd.read_csv("./merge/서울시 지역아동센터 행정동코드 수정.csv", encoding='utf-8')

# 위도/경도 및 Point 컬럼 생성
center_df['lon'] = center_df['경도']
center_df['lat'] = center_df['위도']
center_df['point'] = center_df.progress_apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# intersects 기반 격자 매핑 함수
def find_grid_index_intersects(pt):
    for idx, row in grid_df.iterrows():
        if row['geometry'].intersects(pt):
            return idx
    return None

center_df['grid_index'] = center_df['point'].progress_apply(find_grid_index_intersects)

# 병합용 딕셔너리 구성
center_df['지역아동센터_정보'] = center_df.apply(lambda row: {
    '시설명': row['시설명'],
    '기본주소': row['기본주소'],
    '위도': row['lat'],
    '경도': row['lon']
}, axis=1)

# 병합 전 컬럼 초기화
grid_df['지역아동센터_정보'] = [[] for _ in range(len(grid_df))]

# 병합 수행
for _, row in center_df.dropna(subset=['grid_index']).iterrows():
    idx = int(row['grid_index'])
    grid_df.at[idx, '지역아동센터_정보'].append(row['지역아동센터_정보'])

# 실제 병합된 격자 수 확인
non_empty_count_centers = sum([1 for i in grid_df['지역아동센터_정보'] if len(i) > 0])
non_empty_count_centers

grid_df.to_csv("./서울시격자_지역아동센터포함.csv", index=False, encoding='utf-8')

100%|████████████████████████████████████████████████████████████████████████████████| 409/409 [01:09<00:00,  5.88it/s]


# 키움센터 매핑

In [149]:
tqdm.pandas()

# 데이터로드
grid_df = pd.read_csv("./서울시격자_지역아동센터포함.csv", encoding='utf-8')
grid_df['geometry'] = grid_df['geometry'].apply(wkt.loads)

kiwoom_df = pd.read_csv("./merge/키움센터_행정동코드매핑_최종완벽병합.csv")
kiwoom_df['lon'] = kiwoom_df['경도']
kiwoom_df['lat'] = kiwoom_df['위도']
kiwoom_df['point'] = kiwoom_df.progress_apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# 매핑 함수
def find_grid_index_buffered(pt):
    buffered = pt.buffer(0.00005)  # 약 5m 반경
    for idx, row in grid_df.iterrows():
        if row['geometry'].intersects(buffered):
            return idx
    return None

# 매핑 수행
kiwoom_df['grid_index'] = kiwoom_df['point'].progress_apply(find_grid_index_buffered)

# 매핑 결과 확인
total_kiwoom = len(kiwoom_df)
mapped_kiwoom = kiwoom_df['grid_index'].notna().sum()
unmapped_kiwoom = total_kiwoom - mapped_kiwoom

total_kiwoom, mapped_kiwoom, unmapped_kiwoom

100%|████████████████████████████████████████████████████████████████████████████████| 269/269 [00:47<00:00,  5.66it/s]


(269, 269, 0)

In [151]:
# 키움센터_정보 컬럼
kiwoom_df['키움센터_정보'] = kiwoom_df.apply(lambda row: {
    '시설명': row['시설명'],
    '기본주소': row['기본주소'],
    '위도': row['lat'],
    '경도': row['lon']
}, axis=1)

# 병합 전 컬럼 초기화
grid_df['키움센터_정보'] = [[] for _ in range(len(grid_df))]

# 병합 수행
for _, row in kiwoom_df.dropna(subset=['grid_index']).iterrows():
    idx = int(row['grid_index'])
    grid_df.at[idx, '키움센터_정보'].append(row['키움센터_정보'])

# 최종 저장
grid_df.to_csv("./서울시격자_키움센터포함.csv", index=False, encoding='utf-8')

In [155]:
mapped_kiwoom_ids = kiwoom_df['grid_index'].dropna().unique()
mapped_count = len(mapped_kiwoom_ids)

# 중복 포함 모든 시설 수
total_facilities_mapped = kiwoom_df['grid_index'].notna().sum()

# 최종 결과
mapped_count, total_facilities_mapped, len(kiwoom_df)

(219, 269, 269)

# 유흥주점 데이터 매핑

In [162]:
tqdm.pandas()

# 데이터 로드
adult_df = pd.read_csv("./merge/유흥주점_위경도_행정동코드_추가_최종.csv", encoding='cp949')

# 제외할 사업장명 리스트
excluded_names = [
    "체리", "궁", "비너스클럽", "모델노래크럽", "갤럭시 노래주점",
    "보물섬", "원남관광나이트", "브이아이피(VIP)", "라이브노래주점", "일번지"
]

# 제외 후 대상 데이터
adult_filtered = adult_df[~adult_df['사업장명'].isin(excluded_names)].copy()

# 총 대상 개수
total_adult_candidates = len(adult_filtered)

# 좌표 기반으로 Point 생성
adult_filtered['lon'] = adult_filtered['경도']
adult_filtered['lat'] = adult_filtered['위도']
adult_filtered['point'] = adult_filtered.apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# 격자 데이터 로드 및 geometry 변환
grid_df = pd.read_csv("./서울시격자_키움센터포함.csv", encoding='utf-8')
grid_df['geometry'] = grid_df['geometry'].apply(wkt.loads)

# buffer 없이 intersect만으로 다시 매핑 시도
def find_grid_index_simple(pt):
    for idx, row in grid_df.iterrows():
        if row['geometry'].intersects(pt):
            return idx
    return None

from tqdm import tqdm
tqdm.pandas()
adult_filtered['grid_index'] = adult_filtered['point'].progress_apply(find_grid_index_simple)

# 매핑된 수 확인
mapped_adult = adult_filtered['grid_index'].notna().sum()
unmapped_adult = total_adult_candidates - mapped_adult

total_adult_candidates, mapped_adult, unmapped_adult

100%|██████████████████████████████████████████████████████████████████████████████| 1733/1733 [04:37<00:00,  6.25it/s]


(1733, 1733, 0)

In [166]:
# 유흥시설_정보 컬럼 생성
adult_filtered['유흥시설_정보'] = adult_filtered.apply(lambda row: {
    '시설명': row['사업장명'],
    '도로명주소': row['도로명주소'],
    '위도': row['lat'],
    '경도': row['lon']
}, axis=1)

# 병합 전 컬럼 초기화
grid_df['유흥시설_정보'] = [[] for _ in range(len(grid_df))]

# 병합 수행
for _, row in adult_filtered.dropna(subset=['grid_index']).iterrows():
    idx = int(row['grid_index'])
    grid_df.at[idx, '유흥시설_정보'].append(row['유흥시설_정보'])

# 최종 저장
grid_df.to_csv("./서울시격자_유흥시설포함.csv", index=False, encoding='utf-8')

# 지하철 & 버스 데이터 매핑

In [170]:
# 데이터
grid_df = pd.read_csv("./서울시격자_유흥시설포함.csv", encoding='utf-8')
grid_df['geometry'] = grid_df['geometry'].apply(wkt.loads)

transit_df = pd.read_csv("./merge/서울시_지하철_버스_행정동코드모두포함.csv", encoding='utf-8')


# 좌표 및 Point 생성
transit_df['lon'] = transit_df['경도']
transit_df['lat'] = transit_df['위도']
transit_df['point'] = transit_df.progress_apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# 매핑 함수 (intersects + buffer 방식)
def find_grid_index_buffered(pt):
    buffered = pt.buffer(0.00005)
    for idx, row in grid_df.iterrows():
        if row['geometry'].intersects(buffered):
            return idx
    return None

transit_df['grid_index'] = transit_df['point'].progress_apply(find_grid_index_buffered)

# 병합용 딕셔너리 생성
transit_df['대중교통_정보'] = transit_df.apply(lambda row: {
    '이름': row['이름'],
    '종류': row['종류'],
    '위도': row['lat'],
    '경도': row['lon']
}, axis=1)

# 병합 전 컬럼 초기화
grid_df['대중교통_정보'] = [[] for _ in range(len(grid_df))]

# 병합 수행
for _, row in transit_df.dropna(subset=['grid_index']).iterrows():
    idx = int(row['grid_index'])
    grid_df.at[idx, '대중교통_정보'].append(row['대중교통_정보'])

# 매핑된 격자 수 확인
non_empty_count_transit = sum([1 for i in grid_df['대중교통_정보'] if len(i) > 0])
non_empty_count_transit

100%|████████████████████████████████████████████████████████████████████████████| 11755/11755 [32:21<00:00,  6.05it/s]


1586

In [190]:
missing = transit_df[transit_df['grid_index'].isna()].copy()

# 2. buffer 확장해서 재매핑
def match_buffered(pt):
    buffered = pt.buffer(0.005)
    for idx, row in grid_df.iterrows():
        if row['geometry'].intersects(buffered):
            return idx
    return None

missing['grid_index'] = missing['point'].apply(match_buffered)

# 3. 재매핑 결과 반영
transit_df.update(missing)

# 4. 매핑된 최종 결과 확인
final_mapped = transit_df['grid_index'].notna().sum()
final_unmapped = len(transit_df) - final_mapped

final_mapped, final_unmapped

(11755, 0)

In [192]:
# 병합용 딕셔너리 생성
transit_df['대중교통_정보'] = transit_df.apply(lambda row: {
    '이름': row['이름'],
    '종류': row['종류'],
    '위도': row['위도'],
    '경도': row['경도']
}, axis=1)

# 병합 전 컬럼 초기화
grid_df['대중교통_정보'] = [[] for _ in range(len(grid_df))]

# 병합 수행
for _, row in transit_df.dropna(subset=['grid_index']).iterrows():
    idx = int(row['grid_index'])
    grid_df.at[idx, '대중교통_정보'].append(row['대중교통_정보'])

# 병합된 격자 저장
grid_df.to_csv("./서울시격자_대중교통포함.csv", index=False, encoding='utf-8')

# 서울시 소득소비 데이터 매핑

In [253]:
tqdm.pandas()

# 파일 경로
grid_path = "./서울시격자_대중교통포함.csv"
income_path = "./merge/서울시_소득소비_2022_2024_통계청코드_좌표추가완료.csv"

# 데이터 불러오기
grid_df = pd.read_csv(grid_path, encoding='utf-8')
income_df = pd.read_csv(income_path, encoding='cp949')

# 컬럼명 정리
income_df = income_df.rename(columns={
    '행정동_코드': '행정동코드',
    '월_평균_소득_금액': '월평균소득',
    '소득_구간_코드': '소득분위',
    '지출_총금액': '총지출'
})

# 분기컬럼 만들기
income_df['분기'] = income_df['기준_년분기_코드'].astype(str).map(lambda x: f"{x[:4]}_{x[4]}Q")

# 자료형 통일
grid_df['행정동코드'] = grid_df['행정동코드'].astype(str)
income_df['행정동코드'] = income_df['행정동코드'].astype(str)

# 피벗테이블
income_pivot = income_df.pivot_table(
    index='행정동코드',
    columns='분기',
    values=['월평균소득', '소득분위', '총지출']
)

# 컬럼명재정리
income_pivot.columns = [f"{quarter}_{kind}" for kind, quarter in income_pivot.columns]
income_pivot.reset_index(inplace=True)

# 머지
merged_df = grid_df.merge(income_pivot, on='행정동코드', how='left')

# 중복컬럼제거
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

# 누락행정동체크
income_codes = set(income_df['행정동코드'].unique())
grid_codes = set(grid_df['행정동코드'].unique())
missing_codes = grid_codes - income_codes
print(f"소득소비에 없는 행정동코드 수: {len(missing_codes)}")
print(f"예시: {list(missing_codes)[:5]}")

# 결측값확인
check_cols = [col for col in merged_df.columns if any(q in col for q in ['2022_', '2023_', '2024_'])]
nan_counts = merged_df[check_cols].isna().sum()
print("NaN 발생 컬럼 수:")
print(nan_counts[nan_counts > 0])

# 저장
output_path = "./서울시격자_소득소비매핑.csv"
merged_df.to_csv(output_path, index=False, encoding='utf-8')
print(f"결과 저장 완료: {output_path}")

소득소비에 없는 행정동코드 수: 3
예시: ['11740525', '11680675', '11740526']
NaN 발생 컬럼 수:
2022_1Q_소득분위     21
2022_2Q_소득분위     21
2022_3Q_소득분위     21
2022_4Q_소득분위     21
2023_1Q_소득분위     21
2023_2Q_소득분위     21
2023_3Q_소득분위     21
2023_4Q_소득분위     21
2024_1Q_소득분위     21
2024_2Q_소득분위     21
2024_3Q_소득분위     21
2024_4Q_소득분위     21
2022_1Q_월평균소득    21
2022_2Q_월평균소득    21
2022_3Q_월평균소득    21
2022_4Q_월평균소득    21
2023_1Q_월평균소득    21
2023_2Q_월평균소득    21
2023_3Q_월평균소득    21
2023_4Q_월평균소득    21
2024_1Q_월평균소득    21
2024_2Q_월평균소득    21
2024_3Q_월평균소득    21
2024_4Q_월평균소득    21
2022_1Q_총지출      21
2022_2Q_총지출      21
2022_3Q_총지출      21
2022_4Q_총지출      21
2023_1Q_총지출      21
2023_2Q_총지출      21
2023_3Q_총지출      21
2023_4Q_총지출      21
2024_1Q_총지출      21
2024_2Q_총지출      21
2024_3Q_총지출      21
2024_4Q_총지출      21
dtype: int64
결과 저장 완료: ./서울시격자_소득소비매핑.csv


### 시각화하여 보간필요 확인

In [10]:
# 데이터 로드
path = "./서울시격자_소득소비매핑.csv"
df = pd.read_csv(path, encoding='utf-8')
df['geometry'] = df['geometry'].apply(wkt.loads)

# 컬럼 불러오기
latest_income_col = '2024_4Q_월평균소득'
tooltip_col = '행정동명'

# 중심점 잡기
center = df.iloc[0]['geometry'].centroid.coords[0][::-1]

# 결측과 아닌 행 나누기
valid_df = df[~df[latest_income_col].isna()].copy()
all_df = df.copy()

# 툴팁용 NaN값 대체
all_df[latest_income_col] = all_df[latest_income_col].fillna('-')

# 색상 범위
min_income = valid_df[latest_income_col].min()
max_income = valid_df[latest_income_col].max()

# 모든 격자의 배경
def background_style(feature):
    return {
        'fillColor': '#f0f0f0',
        'color': 'lightgray',
        'weight': 0.3,
        'fillOpacity': 0.1
    }

# 소득 스타일
def income_style(feature):
    value = feature['properties'][latest_income_col]
    if value in [None, '-', '']:
        return {'fillOpacity': 0.0, 'weight': 0}
    else:
        value = float(value)
        color = plt.cm.Oranges((value - min_income) / (max_income - min_income))
        color_hex = matplotlib.colors.rgb2hex(color)
        return {
            'fillColor': color_hex,
            'color': 'gray',
            'weight': 0.3,
            'fillOpacity': 0.7
        }

# geojson 변환
def to_geojson(df, props):
    features = []
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {k: row[k] for k in props}
        }
        features.append(feature)
    return {'type': 'FeatureCollection', 'features': features}

# 지도
m = folium.Map(location=center, zoom_start=11, tiles='cartodbpositron')

# 배경과 툴팁
background_geojson = to_geojson(all_df, [tooltip_col, latest_income_col])
folium.GeoJson(
    background_geojson,
    style_function=background_style,
    tooltip=folium.GeoJsonTooltip(
        fields=[tooltip_col, latest_income_col],
        aliases=["행정동명", "월평균소득 (원)"],
        localize=True,
        sticky=True
    ),
    name="전체 격자 (배경 + 툴팁)"
).add_to(m)

# 소득 색상
income_geojson = to_geojson(valid_df, [tooltip_col, latest_income_col])
folium.GeoJson(
    income_geojson,
    style_function=income_style,
    tooltip=folium.GeoJsonTooltip(
        fields=[tooltip_col, latest_income_col],
        aliases=["행정동명", "월평균소득 (원)"],
        localize=True,
        sticky=True
    ),
    name="소득 있는 격자"
).add_to(m)

# 저장
output_path = "./서울시격자_소득지도_2024_4Q_보간확인.html"
m.save(output_path)
print(f"완료: {output_path}")

완료: ./서울시격자_소득지도_2024_4Q_보간확인.html


### 상일제1동, 상일제2동, 개포3동 주변 행정동들로 평균치 보간하기

In [18]:
# 데이터 드드
df = pd.read_csv("./서울시격자_소득소비매핑.csv", encoding='utf-8')

# 보간 대상 불러오기
interpolation_targets = {
    '상일제2동': ['강일동', '고덕2동'],
    '상일제1동': ['고덕2동', '고덕1동', '명일2동'],
    '개포3동': ['잠실본동', '대치2동', '개포2동', '일원본동', '일원1동', '삼전동']
}

# 컬럼 정렬
income_cols = sorted([col for col in df.columns if col.endswith("월평균소득")])
rank_cols = sorted([col for col in df.columns if col.endswith("소득분위")])
expense_cols = sorted([col for col in df.columns if col.endswith("총지출")])

quarter_columns = list(zip(income_cols, rank_cols, expense_cols))

# 보간
for target_dong, ref_dongs in interpolation_targets.items():
    ref_df = df[df['행정동명'].isin(ref_dongs)]
    for income_col, rank_col, expense_col in quarter_columns:
        df.loc[df['행정동명'] == target_dong, income_col] = ref_df[income_col].mean().round(0)
        df.loc[df['행정동명'] == target_dong, rank_col] = ref_df[rank_col].mean().round(0)
        df.loc[df['행정동명'] == target_dong, expense_col] = ref_df[expense_col].mean().round(0)

# 저장
df.to_csv("./서울시격자_소득소비매핑보간완료.csv", index=False, encoding='utf-8')
print("완료")

완료


### 시각화

In [20]:
# 1. CSV 파일 불러오기
file_path = "서울시격자_소득소비매핑보간완료.csv"
df = pd.read_csv(file_path, encoding='utf-8')

# 2. geometry 컬럼 WKT → shapely Polygon
df["geometry"] = df["geometry"].apply(wkt.loads)

# 3. 소득/지출 컬럼 정의
df["월평균_소득"] = df["2024_4Q_월평균소득"]
df["월평균_총지출"] = df["2024_4Q_총지출"]
df["격자ID"] = df["grid_id"]  # Choropleth용
df["행정동명"] = df["행정동명"]  # 툴팁용

# 4. 총지출 컬럼 클리핑 (이상치 제거: 1~99 분위)
vmin_spend = np.percentile(df["월평균_총지출"], 1)
vmax_spend = np.percentile(df["월평균_총지출"], 99)
df["클리핑_총지출"] = df["월평균_총지출"].clip(lower=vmin_spend, upper=vmax_spend)

# 5. GeoJSON 생성 함수
def df_to_geojson(dataframe):
    features = []
    for _, row in dataframe.iterrows():
        features.append({
            "type": "Feature",
            "geometry": mapping(row["geometry"]),
            "properties": {
                "행정동명": row["행정동명"],
                "월평균_소득": row["월평균_소득"],
                "월평균_총지출": row["월평균_총지출"],
                "클리핑_총지출": row["클리핑_총지출"],
                "격자ID": row["격자ID"]
            }
        })
    return {
        "type": "FeatureCollection",
        "features": features
    }

geojson_data = df_to_geojson(df)

# 6. Folium 지도 생성 함수
def draw_folium_map_custom(column, legend_name):
    m = folium.Map(location=[37.5665, 126.9780], zoom_start=11, tiles='cartodbpositron')
    
    Choropleth(
        geo_data=geojson_data,
        data=df,
        columns=["격자ID", column],
        key_on="feature.properties.격자ID",
        fill_color="YlOrRd",
        fill_opacity=0.7,
        line_opacity=0.3,
        line_color="black",
        nan_fill_color="white",
        legend_name=legend_name
    ).add_to(m)

    GeoJson(
        geojson_data,
        tooltip=GeoJsonTooltip(
            fields=["행정동명", "월평균_소득", "월평균_총지출"],
            aliases=["행정동", "월평균 소득", "월평균 총지출"],
            localize=True
        ),
        style_function=lambda x: {
            "color": "black",
            "weight": 0.5,
            "fillOpacity": 0
        }
    ).add_to(m)

    return m

# 7. 지도 생성
income_map = draw_folium_map_custom("월평균_소득", "2024년 4분기 월평균 소득")
spending_map = draw_folium_map_custom("클리핑_총지출", "2024년 4분기 월평균 총지출 (클리핑)")

# 8. HTML로 저장
income_map.save("./시각화파일모음/2024Q4_소득_지도.html")
spending_map.save("./시각화파일모음/2024Q4_총지출_지도_클리핑.html")

In [27]:
file_path = "서울시격자_소득소비매핑보간완료.csv"
df = pd.read_csv(file_path, encoding='utf-8')

# '총지출'이 포함된 컬럼 전체 추출
spending_columns = [col for col in df.columns if "총지출" in col and "202" in col]

# GeoJSON 생성: 총지출 관련 컬럼 전부 포함
def geojson_all_spending(dataframe, spending_cols):
    features = []
    for _, row in dataframe.iterrows():
        geom = row["geometry"]
        if isinstance(geom, str):
            geom = wkt.loads(geom)
        props = {"행정동명": row["행정동명"]}
        for col in spending_cols:
            props[col] = row[col]
        features.append({
            "type": "Feature",
            "geometry": mapping(geom),
            "properties": props
        })
    return {
        "type": "FeatureCollection",
        "features": features
    }

# GeoJSON 생성
kepler_spending_geojson = geojson_all_spending(df, spending_columns)

# 저장
output_path = "./시각화용파일/서울시_총지출_전분기_Kepler.geojson"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(kepler_spending_geojson, f, ensure_ascii=False)

output_path

'./시각화용파일/서울시_총지출_전분기_Kepler.geojson'

# 인구정보 매핑하기 - 생활인구 매핑하기

# 아동인구 매핑하기

In [35]:
# 파일 불러오기
grid = pd.read_csv('./서울시격자_소득소비매핑보간완료.csv')
child = pd.read_csv('./merge/행정동별_아동인구_코드포함_최종병합_소계제외.csv', encoding='cp949')
child = child.rename(columns={'행자부행정동코드': '행정동코드'})

# 거주 가능 키워드 정의
residential_keywords = [
    "주거지역", "상업지역", "공업지역", 
    "전용주거지역", "준공업지역", "근린상업지역", "복합지역"
]

# 거주 가능한 격자 필터링
residential_grid = grid[grid['지적편집도_상세정보'].fillna('').apply(
    lambda x: any(k in x for k in residential_keywords)
)].copy()

# 분배비율 계산 (동별 격자 수 기준)
residential_grid['분배비율'] = 1 / residential_grid.groupby('행정동코드')['grid_id'].transform('count')

# 동별 분배비율만 추출
ratio_df = residential_grid[['grid_id', '행정동코드', '분배비율']].copy()

# 분배 대상 아동인구만 추출
age_cols = [col for col in child.columns if col.startswith(('2022', '2023', '2024'))]
child_subset = child[['행정동코드'] + age_cols]

# 분배 데이터프레임 병합
merged = pd.merge(ratio_df, child_subset, on='행정동코드', how='left')

# 아동인구 분배
for col in age_cols:
    merged[col] = (merged[col] * merged['분배비율']).round().astype('Int64')

# 원본 grid에 병합
final_grid = grid.copy()
final_grid = pd.merge(final_grid, merged[['grid_id'] + age_cols], on='grid_id', how='left')

# 빈 곳은 0으로 채움
for col in age_cols:
    final_grid[col] = final_grid[col].fillna(0).astype('Int64')

# 저장
final_grid.to_csv('서울시격자_아동인구매핑완료.csv', index=False, encoding='utf-8')

In [37]:
final_grid.isna().sum()

grid_id          0
geometry         0
행정동명             0
행정동코드            0
지적편집도_상세정보       0
지적편집도_상위정보       0
서울형공립키즈카페_정보     0
인증제키즈카페_정보       0
일반키즈카페_정보        0
유치원_초등학교_정보      0
어린이집_정보          0
지역아동센터_정보        0
키움센터_정보          0
유흥시설_정보          0
대중교통_정보          0
2022_1Q_소득분위     0
2022_2Q_소득분위     0
2022_3Q_소득분위     0
2022_4Q_소득분위     0
2023_1Q_소득분위     0
2023_2Q_소득분위     0
2023_3Q_소득분위     0
2023_4Q_소득분위     0
2024_1Q_소득분위     0
2024_2Q_소득분위     0
2024_3Q_소득분위     0
2024_4Q_소득분위     0
2022_1Q_월평균소득    0
2022_2Q_월평균소득    0
2022_3Q_월평균소득    0
2022_4Q_월평균소득    0
2023_1Q_월평균소득    0
2023_2Q_월평균소득    0
2023_3Q_월평균소득    0
2023_4Q_월평균소득    0
2024_1Q_월평균소득    0
2024_2Q_월평균소득    0
2024_3Q_월평균소득    0
2024_4Q_월평균소득    0
2022_1Q_총지출      0
2022_2Q_총지출      0
2022_3Q_총지출      0
2022_4Q_총지출      0
2023_1Q_총지출      0
2023_2Q_총지출      0
2023_3Q_총지출      0
2023_4Q_총지출      0
2024_1Q_총지출      0
2024_2Q_총지출      0
2024_3Q_총지출      0
2024_4Q_총지출      0
2022_0-4세        0
2022_5-9세   

In [39]:
# 결측이 거주불가능 격자 수와 일치하는지 확인
전체_격자수 = len(grid)

# 아동인구가 할당된 격자 수 (예: 2022_0-4세 기준)
할당된_격자수 = final_grid['2022_0-4세'].notna().sum()

# 결측치 수
결측치_수 = final_grid['2022_0-4세'].isna().sum()

print(f'전체 격자 수: {전체_격자수}')
print(f'아동인구 할당된 격자 수: {할당된_격자수}')
print(f'결측치 수: {결측치_수}')

전체 격자 수: 2427
아동인구 할당된 격자 수: 2427
결측치 수: 0


### 나중에 아동인구 fill_na=0 필요한 경우 코드 찾기

In [43]:
final_grid.fillna({
    '2022_0-4세': 0, '2022_5-9세': 0,
    '2023_0-4세': 0, '2023_5-9세': 0,
    '2024_0-4세': 0, '2024_5-9세': 0
}, inplace=True)

In [29]:
pd.set_option('display.max_rows', None)

### 매핑 오차 확인

In [ ]:
df = pd.read_csv('./서울시격자_아동인구매핑완료.csv')

# 동별 분배 합계 확인
summary = df.groupby('행정동코드')[['2022_0-4세', '2022_5-9세']].sum().astype('Int64')

# 원본 아동인구 데이터 비교
original = child.set_index('행정동코드')[['2022_0-4세', '2022_5-9세']]

# 차이 확인
diff = summary.subtract(original, fill_value=0)
print(diff[abs(diff) > 0].dropna(how='all'))

### 시각화용 파일 처리

In [47]:
# 분배 완료된 격자 데이터 불러오기
df = pd.read_csv("./서울시격자_아동인구매핑완료.csv")

# geometry 컬럼이 WKT 형태인 경우 처리
if 'geometry' in df.columns:
    df['geometry'] = df['geometry'].apply(wkt.loads)
else:
    raise ValueError("geometry 컬럼이 없어요. 중심좌표 계산이 안 됩니다.")

# 중심좌표 계산
df['lon'] = df['geometry'].apply(lambda x: x.centroid.x)
df['lat'] = df['geometry'].apply(lambda x: x.centroid.y)

# Kepler용 저장: geometry 제거하고 중심좌표 포함
cols_to_export = ['grid_id', 'lon', 'lat'] + [col for col in df.columns if col.startswith(('2022', '2023', '2024'))]
df_kepler = df[cols_to_export]

# 저장
df_kepler.to_csv("./시각화용파일/서울시격자_아동인구_Kepler용.csv", index=False, encoding='utf-8')

# 생활인구 2022, 2023, 2024 매핑하기

### 202201생활인구 매핑 테스트 -> 성공!

In [9]:
# 데이터 로딩 및 전리리
people_df = pd.read_csv("G:\내 드라이브\DIMA\프로젝트\생활인구데이터\LOCAL_PEOPLE_DONG_202201.csv", index_col=False, encoding='utf-8')
grid_df = pd.read_csv("./서울시격자_아동인구매핑완료.csv", encoding='utf-8')
poi_df = pd.read_csv("./merge/서울시_관광특구_POI_좌표.csv", encoding='cp949')

people_df['행정동코드'] = people_df['행정동코드'].astype(float).astype(int).astype(str)
grid_df['행정동코드'] = grid_df['행정동코드'].astype(str)
poi_coords = list(zip(poi_df['latitude'], poi_df['longitude']))

# 거주 가능 격자 필터링 및 가중치 계산
keywords = ['주거지역', '상업지역', '공업지역', '전용주거지역', '준공업지역', '근린상업지역', '복합지역']

def contains_residential(info):
    try:
        d = ast.literal_eval(info)
        return any(any(k in key for k in keywords) for key in d)
    except:
        return False

def calc_distance_weight(lat, lon):
    grid_point = (lat, lon)
    return sum(1 / max(haversine(grid_point, poi), 0.001) for poi in poi_coords)

def calc_centroid(geom):
    try:
        poly = wkt.loads(geom)
        return poly.centroid.y, poly.centroid.x
    except:
        return None, None

res_grids = grid_df[grid_df['지적편집도_상세정보'].apply(contains_residential)].copy()
res_grids[['lat', 'lon']] = res_grids['geometry'].apply(lambda g: pd.Series(calc_centroid(g)))
res_grids['거주가중치'] = 1.0  # 실제 거주 비율이 있다면 이 부분에 넣어줘
res_grids['거리가중치'] = res_grids.apply(lambda row: calc_distance_weight(row['lat'], row['lon']), axis=1)
res_grids['최종가중치'] = res_grids['거주가중치'] * res_grids['거리가중치']

# 보간 매핑
보간매핑 = {
    '11305600': ['11305615', '11305625', '11305635'],  # 수유동 → 수유1~3동
    '11305590': ['11305595'],                          # 번동 → 번1동
    '11740520': ['11740525', '11740526']               # 상일동 → 상일제1,2동
}

# 보간 매핑 + 일반 동 포함한 전체 매핑
전체매핑 = {**보간매핑, **{c: [c] for c in people_df['행정동코드'].unique() if c not in 보간매핑}}

# 분배배
grid_values = defaultdict(lambda: defaultdict(float))

for _, row in tqdm(people_df.iterrows(), total=len(people_df)):
    date_id = str(row['기준일ID'])
    hour = int(row['시간대구분'])
    col_name = f"{date_id}_{hour:02d}"
    hd_code = row['행정동코드']
    pop = float(row['총생활인구수'])

    target_codes = 전체매핑[hd_code]
    sub_grids = res_grids[res_grids['행정동코드'].isin(target_codes)]

    if sub_grids.empty:
        continue

    weights = sub_grids['최종가중치'].values
    weight_sum = weights.sum()
    if weight_sum == 0:
        continue

    ratios = weights / weight_sum
    for grid_id, ratio in zip(sub_grids['grid_id'], ratios):
        grid_values[grid_id][col_name] += pop * ratio

# wide 포맷 변환, 저장, 지지
all_columns = sorted({col for d in grid_values.values() for col in d})
all_grid_ids = sorted(grid_values.keys())

wide_df = pd.DataFrame(index=all_grid_ids, columns=all_columns)
for grid_id in tqdm(all_grid_ids, desc="Building wide DataFrame"):
    for col in all_columns:
        wide_df.at[grid_id, col] = grid_values[grid_id].get(col, 0)

wide_df.reset_index(inplace=True)
wide_df.rename(columns={'index': 'grid_id'}, inplace=True)

# 격자 정보와 머지하여 저장
final_df = pd.merge(grid_df, wide_df, on="grid_id", how="inner")
final_df.to_csv("./서울시격자_202201.csv", index=False, encoding='utf-8')
print("완료")

<>:13: SyntaxWarning: invalid escape sequence '\D'
<>:13: SyntaxWarning: invalid escape sequence '\D'
C:\Users\master\AppData\Local\Temp\ipykernel_12556\1665302106.py:13: SyntaxWarning: invalid escape sequence '\D'
  people_df = pd.read_csv("G:\내 드라이브\DIMA\프로젝트\생활인구데이터\LOCAL_PEOPLE_DONG_202201.csv", index_col=False, encoding='utf-8')
Building wide DataFrame: 100%|████████████████████████████████████████████████████| 2034/2034 [00:19<00:00, 102.15it/s]


완료


In [ ]:
def extract_centroid_coords(geom_str):
    try:
        polygon = wkt.loads(geom_str)
        centroid = polygon.centroid
        return pd.Series({'lon': centroid.x, 'lat': centroid.y})
    except:
        return pd.Series({'lon': None, 'lat': None})

final_df[['lon', 'lat']] = final_df['geometry'].apply(extract_centroid_coords)

# 시간 컬럼 필터링
time_columns = [col for col in final_df.columns if re.fullmatch(r'20\d{6}_\d{2}', col)]

# wide → long
long_df = pd.melt(
    final_df,
    id_vars=['grid_id', 'lat', 'lon'],
    value_vars=time_columns,
    var_name='time',
    value_name='인구수'
)
long_df['datetime'] = pd.to_datetime(long_df['time'], format='%Y%m%d_%H')
kepler_df = long_df[['grid_id', 'lat', 'lon', 'datetime', '인구수']]
kepler_df.to_csv("격자생활인구_Kepler_202201.csv", index=False)
print("Kepler 시각화용 CSV 저장 완료")

### 2022년 모두 저장하기

In [2]:
# 기본정보
keyword_list = ['주거지역', '상업지역', '공업지역', '전용주거지역', '준공업지역', '근린상업지역', '복합지역']
poi_df = pd.read_csv("./merge/서울시_관광특구_POI_좌표.csv", encoding='cp949')
grid_df = pd.read_csv("./서울시격자_아동인구매핑완료.csv", encoding='utf-8')
grid_df['행정동코드'] = grid_df['행정동코드'].astype(str)
poi_coords = list(zip(poi_df['latitude'], poi_df['longitude']))

# 공통함수
def contains_residential(info):
    try:
        d = ast.literal_eval(info)
        return any(any(k in key for k in keyword_list) for key in d)
    except:
        return False

def calc_distance_weight(lat, lon):
    grid_point = (lat, lon)
    return sum(1 / max(haversine(grid_point, poi), 0.001) for poi in poi_coords)

def calc_centroid(geom):
    try:
        poly = wkt.loads(geom)
        return poly.centroid.y, poly.centroid.x
    except:
        return None, None

def extract_centroid_coords(geom_str):
    try:
        polygon = wkt.loads(geom_str)
        centroid = polygon.centroid
        return pd.Series({'lon': centroid.x, 'lat': centroid.y})
    except:
        return pd.Series({'lon': None, 'lat': None})

# 1년 루프
for yyyymm in [f"2022{m:02d}" for m in range(1, 13)]:
    print(f"처리 중: {yyyymm}")

    path = f"G:\\내 드라이브\\DIMA\\프로젝트\\생활인구데이터\\LOCAL_PEOPLE_DONG_{yyyymm}.csv"
    if not os.path.exists(path):
        print(f"{path} 파일 없음, 건너뜀")
        continue

    # 생활인구 로딩 및 전처리
    people_df = pd.read_csv(path, index_col=False, encoding='utf-8')
    people_df['행정동코드'] = people_df['행정동코드'].astype(float).astype(int).astype(str)

    # 2. 보간 매핑 정의
    보간매핑 = {
        '11305600': ['11305615', '11305625', '11305635'],
        '11305590': ['11305595'],
        '11740520': ['11740525', '11740526']
    }
    전체매핑 = {**보간매핑, **{c: [c] for c in people_df['행정동코드'].unique() if c not in 보간매핑}}

    # 거주 가능 격자 필터링 + 가중치 계산
    res_grids = grid_df[grid_df['지적편집도_상세정보'].apply(contains_residential)].copy()
    res_grids[['lat', 'lon']] = res_grids['geometry'].apply(lambda g: pd.Series(calc_centroid(g)))
    res_grids['거주가중치'] = 1.0
    res_grids['거리가중치'] = res_grids.apply(lambda row: calc_distance_weight(row['lat'], row['lon']), axis=1)
    res_grids['최종가중치'] = res_grids['거주가중치'] * res_grids['거리가중치']

    # 인구 분배
    grid_values = defaultdict(lambda: defaultdict(float))
    for _, row in tqdm(people_df.iterrows(), total=len(people_df)):
        date_id = str(row['기준일ID'])
        hour = int(row['시간대구분'])
        col_name = f"{date_id}_{hour:02d}"
        hd_code = row['행정동코드']
        pop = float(row['총생활인구수'])

        target_codes = 전체매핑.get(hd_code, [hd_code])
        sub_grids = res_grids[res_grids['행정동코드'].isin(target_codes)]

        if sub_grids.empty:
            continue

        weights = sub_grids['최종가중치'].values
        weight_sum = weights.sum()
        if weight_sum == 0:
            continue

        ratios = weights / weight_sum
        for grid_id, ratio in zip(sub_grids['grid_id'], ratios):
            grid_values[grid_id][col_name] += pop * ratio

    # wide 포맷 저장 (모든 격자 유지)
    all_columns = sorted({col for d in grid_values.values() for col in d})
    wide_df = pd.DataFrame(index=grid_df['grid_id'], columns=all_columns, dtype=float).fillna(0.0)

    for grid_id in tqdm(grid_values, desc=f"{yyyymm} wide 생성"):
        for col, value in grid_values[grid_id].items():
            wide_df.at[grid_id, col] = float(value)

    wide_df.reset_index(inplace=True)
    wide_df.rename(columns={'index': 'grid_id'}, inplace=True)

    # 격자와 병합 + 저장 (left join)
    final_df = pd.merge(grid_df, wide_df, on="grid_id", how="left").fillna(0)

    # 최종 저장 전 보간 처리 코드
    interpolation_target_dongs = {
        '항동': ['오류2동', '수궁동'],
        '개포3동': ['잠실본동', '대치2동', '개포2동', '일원본동', '일원1동', '삼전동'],
        '번2동': ['창3동', '미아동', '번1동'],
        '번3동': ['장위1동', '장위3동', '월계2동', '송중동'],
        '상일2동': ['강일동', '고덕2동'],
        '상일1동': ['고덕2동', '고덕1동', '명일2동'],
        '제기동': ['청량리동', '용신동'],
        '홍제1동': ['홍제2동', '홍제3동'],
        '홍제2동': ['홍제1동', '홍제3동'],
        '홍제3동': ['홍제1동', '홍제2동']
    }
    
    # 생활인구 컬럼 정의 (날짜_시간대 형식)
    pop_columns = [col for col in final_df.columns if re.match(r'20\d{6}_\d{2}', col)]
    
    # 거주 가능하지만 생활인구가 0인 격자 확인
    final_df['총생활인구수'] = final_df[pop_columns].sum(axis=1)
    no_pop_grids = final_df[
        (final_df['총생활인구수'] == 0) &
        final_df['지적편집도_상세정보'].apply(contains_residential)
    ]
    
    print(f"보간이 필요한 격자 수: {len(no_pop_grids)}")
    
    # 보간 처리 수행
    for idx, row in tqdm(no_pop_grids.iterrows(), total=len(no_pop_grids), desc="생활인구 보간"):
        target_dong = row['행정동명']
        ref_dongs = interpolation_target_dongs.get(target_dong, [])
        
        if not ref_dongs:
            continue
        
        ref_grids = final_df[final_df['행정동명'].isin(ref_dongs) & (final_df['총생활인구수'] > 0)]
        
        if ref_grids.empty:
            continue
        
        # 참조 동의 평균 생활인구를 구해 보간
        mean_pop = ref_grids[pop_columns].mean()
        final_df.loc[idx, pop_columns] = mean_pop.values
        final_df.loc[idx, '총생활인구수'] = mean_pop.sum()

    # 최종 저장
    final_df.drop(columns=['총생활인구수'], inplace=True)
    final_df.to_csv(f"./서울시격자_{yyyymm}_보간완료.csv", index=False)
    print(f"{yyyymm} 보간완료 저장 완료!")

처리 중: 202201


202201 wide 생성: 100%|████████████████████████████████████████████████████████████| 2034/2034 [00:26<00:00, 78.22it/s]


보간이 필요한 격자 수: 20


생활인구 보간: 100%|███████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  8.99it/s]


202201 보간완료 저장 완료!
처리 중: 202202


...

202211 보간완료 저장 완료!
처리 중: 202212


202212 wide 생성: 100%|████████████████████████████████████████████████████████████| 2034/2034 [00:26<00:00, 77.48it/s]


보간이 필요한 격자 수: 20


생활인구 보간: 100%|███████████████████████████████████████████████████████████████████| 20/20 [00:03<00:00,  5.99it/s]


202212 보간완료 저장 완료!


### 거주가능 격자 중 누락 있는지 확인

In [3]:
# 키워드와 함수 정의
keyword_list = ['주거지역', '상업지역', '공업지역', '전용주거지역', '준공업지역', '근린상업지역', '복합지역']
grid_df = pd.read_csv("./서울시격자_아동인구매핑완료.csv", encoding='utf-8')
grid_df['행정동코드'] = grid_df['행정동코드'].astype(str)

def contains_residential(info):
    try:
        d = ast.literal_eval(info)
        return any(any(k in key for k in keyword_list) for key in d)
    except:
        return False

# 거주 가능 격자 ID 추출
res_grids = grid_df[grid_df['지적편집도_상세정보'].apply(contains_residential)]
residential_ids = set(res_grids['grid_id'])

# 루프 돌면서 누락 확인
for yyyymm in [f"2022{m:02d}" for m in range(1, 13)]:
    print(f"\n{yyyymm} 검사 중")

    merged_path = f"서울시격자_{yyyymm}_보간완료.csv"
    if not os.path.exists(merged_path):
        print(f" - 파일 없음: {merged_path}")
        continue

    merged_df = pd.read_csv(merged_path)

    # 인구 관련 열 추출
    pop_cols = [col for col in merged_df.columns if re.fullmatch(r'20\d{6}_\d{2}', col)]
    merged_df['인구합'] = merged_df[pop_cols].sum(axis=1)

    # 인구합이 0인 격자
    zero_pop_grids = merged_df[merged_df['인구합'] == 0]['grid_id'].tolist()

    # 거주 가능한데 인구가 0인 격자만 필터링
    missed = [gid for gid in zero_pop_grids if gid in residential_ids]

    print(f" - 거주 가능 격자 중 인구 0인 격자 수: {len(missed)}")
    if missed:
        print(f"   예시: {missed[:5]}")


202201 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202202 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202203 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202204 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202205 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202206 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202207 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202208 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202209 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202210 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202211 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202212 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0


### 데이터 매핑 수치 확인

In [ ]:
for yyyymm in [f"2022{m:02d}" for m in range(1, 13)]:
    print(f"\n{yyyymm} 비교 중")

    pred_path = f"./서울시격자_{yyyymm}_보간완료.csv"
    true_path = f"G:\\내 드라이브\\DIMA\\프로젝트\\생활인구데이터\\LOCAL_PEOPLE_DONG_{yyyymm}.csv"

    if not os.path.exists(pred_path) or not os.path.exists(true_path):
        print(f" - 파일 누락: {pred_path if not os.path.exists(pred_path) else true_path}")
        continue

    # 파일 로딩
    pred_df = pd.read_csv(pred_path, encoding='utf-8')
    true_df = pd.read_csv(true_path, encoding='utf-8', index_col=False)

    # 정제
    pred_df['행정동코드'] = pred_df['행정동코드'].astype(str)
    true_df['행정동코드'] = true_df['행정동코드'].astype(float).astype(int).astype(str)
    true_df['colname'] = true_df.apply(lambda r: f"{r['기준일ID']}_{int(r['시간대구분']):02d}", axis=1)

    # 시간 필터
    time_columns = [col for col in pred_df.columns if col.startswith("2022") and len(col) == 11]

    # 집계 및 병합
    pred_by_dong = pred_df.groupby("행정동코드")[time_columns].sum().reset_index()
    true_pivot = true_df.pivot_table(index="행정동코드", columns="colname", values="총생활인구수", aggfunc="sum").reset_index()
    merged = pd.merge(pred_by_dong, true_pivot, on="행정동코드", suffixes=("_예측", "_실제"))

    # 비교 지표 계산
    metrics = []
    for col in time_columns:
        col_pred = f"{col}_예측"
        col_true = f"{col}_실제"
        if col_pred not in merged.columns or col_true not in merged.columns:
            continue

        pred = merged[col_pred]
        true = merged[col_true]
        diff = (pred - true).abs()
        mae = diff.mean()
        rmse = np.sqrt(((pred - true) ** 2).mean())
        total_diff = (pred.sum() - true.sum()) / true.sum() * 100

        metrics.append({
            "시간대": col,
            "MAE": mae,
            "RMSE": rmse,
            "총량오차(%)": total_diff
        })

    month_result_df = pd.DataFrame(metrics)
    print(month_result_df.head(24))

### 2023년 모두 저장하기

In [6]:
# 기본정보
keyword_list = ['주거지역', '상업지역', '공업지역', '전용주거지역', '준공업지역', '근린상업지역', '복합지역']
poi_df = pd.read_csv("./merge/서울시_관광특구_POI_좌표.csv", encoding='cp949')
grid_df = pd.read_csv("./서울시격자_아동인구매핑완료.csv", encoding='utf-8')
grid_df['행정동코드'] = grid_df['행정동코드'].astype(str)
poi_coords = list(zip(poi_df['latitude'], poi_df['longitude']))

# 공통함수
def contains_residential(info):
    try:
        d = ast.literal_eval(info)
        return any(any(k in key for k in keyword_list) for key in d)
    except:
        return False

def calc_distance_weight(lat, lon):
    grid_point = (lat, lon)
    return sum(1 / max(haversine(grid_point, poi), 0.001) for poi in poi_coords)

def calc_centroid(geom):
    try:
        poly = wkt.loads(geom)
        return poly.centroid.y, poly.centroid.x
    except:
        return None, None

def extract_centroid_coords(geom_str):
    try:
        polygon = wkt.loads(geom_str)
        centroid = polygon.centroid
        return pd.Series({'lon': centroid.x, 'lat': centroid.y})
    except:
        return pd.Series({'lon': None, 'lat': None})

# 1년 루프
for yyyymm in [f"2023{m:02d}" for m in range(1, 13)]:
    print(f"처리 중: {yyyymm}")

    path = f"G:\\내 드라이브\\DIMA\\프로젝트\\생활인구데이터\\LOCAL_PEOPLE_DONG_{yyyymm}.csv"
    if not os.path.exists(path):
        print(f"{path} 파일 없음, 건너뜀")
        continue

    # 생활인구 로딩 및 전처리
    people_df = pd.read_csv(path, index_col=False, encoding='utf-8')
    people_df['행정동코드'] = people_df['행정동코드'].astype(float).astype(int).astype(str)

    # 보간 매핑 정의
    보간매핑 = {
        '11305600': ['11305615', '11305625', '11305635'],
        '11305590': ['11305595'],
        '11740520': ['11740525', '11740526']
    }
    전체매핑 = {**보간매핑, **{c: [c] for c in people_df['행정동코드'].unique() if c not in 보간매핑}}

    # 거주 가능 격자 필터링 + 가중치 계산
    res_grids = grid_df[grid_df['지적편집도_상세정보'].apply(contains_residential)].copy()
    res_grids[['lat', 'lon']] = res_grids['geometry'].apply(lambda g: pd.Series(calc_centroid(g)))
    res_grids['거주가중치'] = 1.0
    res_grids['거리가중치'] = res_grids.apply(lambda row: calc_distance_weight(row['lat'], row['lon']), axis=1)
    res_grids['최종가중치'] = res_grids['거주가중치'] * res_grids['거리가중치']

    # 인구 분배
    grid_values = defaultdict(lambda: defaultdict(float))
    for _, row in tqdm(people_df.iterrows(), total=len(people_df)):
        date_id = str(row['기준일ID'])
        hour = int(row['시간대구분'])
        col_name = f"{date_id}_{hour:02d}"
        hd_code = row['행정동코드']
        pop = float(row['총생활인구수'])

        target_codes = 전체매핑.get(hd_code, [hd_code])
        sub_grids = res_grids[res_grids['행정동코드'].isin(target_codes)]

        if sub_grids.empty:
            continue

        weights = sub_grids['최종가중치'].values
        weight_sum = weights.sum()
        if weight_sum == 0:
            continue

        ratios = weights / weight_sum
        for grid_id, ratio in zip(sub_grids['grid_id'], ratios):
            grid_values[grid_id][col_name] += pop * ratio

    # wide 포맷 저장 (모든 격자 유지)
    all_columns = sorted({col for d in grid_values.values() for col in d})
    wide_df = pd.DataFrame(index=grid_df['grid_id'], columns=all_columns, dtype=float).fillna(0.0)

    for grid_id in tqdm(grid_values, desc=f"{yyyymm} wide 생성"):
        for col, value in grid_values[grid_id].items():
            wide_df.at[grid_id, col] = float(value)

    wide_df.reset_index(inplace=True)
    wide_df.rename(columns={'index': 'grid_id'}, inplace=True)

    # 격자와 병합 + 저장 (left join)
    final_df = pd.merge(grid_df, wide_df, on="grid_id", how="left").fillna(0)

    # 최종 저장 전 보간 처리 코드
    interpolation_target_dongs = {
        '항동': ['오류2동', '수궁동'],
        '개포3동': ['잠실본동', '대치2동', '개포2동', '일원본동', '일원1동', '삼전동'],
        '번2동': ['창3동', '미아동', '번1동'],
        '번3동': ['장위1동', '장위3동', '월계2동', '송중동'],
        '상일2동': ['강일동', '고덕2동'],
        '상일1동': ['고덕2동', '고덕1동', '명일2동'],
        '제기동': ['청량리동', '용신동'],
        '홍제1동': ['홍제2동', '홍제3동'],
        '홍제2동': ['홍제1동', '홍제3동'],
        '홍제3동': ['홍제1동', '홍제2동']
    }
    
    # 생활인구 컬럼 정의 (날짜_시간대 형식)
    pop_columns = [col for col in final_df.columns if re.match(r'20\d{6}_\d{2}', col)]
    
    # 거주 가능하지만 생활인구가 0인 격자 확인
    final_df['총생활인구수'] = final_df[pop_columns].sum(axis=1)
    no_pop_grids = final_df[
        (final_df['총생활인구수'] == 0) &
        final_df['지적편집도_상세정보'].apply(contains_residential)
    ]
    
    print(f"보간이 필요한 격자 수: {len(no_pop_grids)}")
    
    # 보간 처리 수행
    for idx, row in tqdm(no_pop_grids.iterrows(), total=len(no_pop_grids), desc="생활인구 보간"):
        target_dong = row['행정동명']
        ref_dongs = interpolation_target_dongs.get(target_dong, [])
        
        if not ref_dongs:
            continue
        
        ref_grids = final_df[final_df['행정동명'].isin(ref_dongs) & (final_df['총생활인구수'] > 0)]
        
        if ref_grids.empty:
            continue
        
        # 참조 동의 평균 생활인구를 구해 보간
        mean_pop = ref_grids[pop_columns].mean()
        final_df.loc[idx, pop_columns] = mean_pop.values
        final_df.loc[idx, '총생활인구수'] = mean_pop.sum()

    # 최종 저장
    final_df.drop(columns=['총생활인구수'], inplace=True)
    final_df.to_csv(f"./서울시격자_{yyyymm}_보간완료.csv", index=False)
    print(f"{yyyymm} 보간완료 저장 완료!")

처리 중: 202301


202301 wide 생성: 100%|████████████████████████████████████████████████████████████| 2034/2034 [00:26<00:00, 77.16it/s]


보간이 필요한 격자 수: 20


생활인구 보간: 100%|███████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  8.32it/s]


202301 보간완료 저장 완료!
처리 중: 202302


...

202311 보간완료 저장 완료!
처리 중: 202312


202312 wide 생성: 100%|████████████████████████████████████████████████████████████| 2034/2034 [00:26<00:00, 75.76it/s]


보간이 필요한 격자 수: 20


생활인구 보간: 100%|███████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  8.12it/s]


202312 보간완료 저장 완료!


### 누락확인

In [7]:
# 키워드와 함수 정의
keyword_list = ['주거지역', '상업지역', '공업지역', '전용주거지역', '준공업지역', '근린상업지역', '복합지역']
grid_df = pd.read_csv("./서울시격자_아동인구매핑완료.csv", encoding='utf-8')
grid_df['행정동코드'] = grid_df['행정동코드'].astype(str)

def contains_residential(info):
    try:
        d = ast.literal_eval(info)
        return any(any(k in key for k in keyword_list) for key in d)
    except:
        return False

# 거주 가능 격자 ID 추출
res_grids = grid_df[grid_df['지적편집도_상세정보'].apply(contains_residential)]
residential_ids = set(res_grids['grid_id'])

# 루프 돌면서 누락 확인
for yyyymm in [f"2023{m:02d}" for m in range(1, 13)]:
    print(f"\n{yyyymm} 검사 중")

    merged_path = f"서울시격자_{yyyymm}_보간완료.csv"
    if not os.path.exists(merged_path):
        print(f" - 파일 없음: {merged_path}")
        continue

    merged_df = pd.read_csv(merged_path)

    # 인구 관련 열 추출
    pop_cols = [col for col in merged_df.columns if re.fullmatch(r'20\d{6}_\d{2}', col)]
    merged_df['인구합'] = merged_df[pop_cols].sum(axis=1)

    # 인구합이 0인 격자
    zero_pop_grids = merged_df[merged_df['인구합'] == 0]['grid_id'].tolist()

    # 거주 가능한데 인구가 0인 격자만 필터링
    missed = [gid for gid in zero_pop_grids if gid in residential_ids]

    print(f" - 거주 가능 격자 중 인구 0인 격자 수: {len(missed)}")
    if missed:
        print(f"   예시: {missed[:5]}")


202301 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202302 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202303 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202304 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202305 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202306 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202307 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202308 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202309 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202310 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202311 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202312 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0


### 매핑수치 확인

In [ ]:
for yyyymm in [f"2023{m:02d}" for m in range(1, 13)]:
    print(f"\n{yyyymm} 비교 중")

    pred_path = f"./서울시격자_{yyyymm}_보간완료.csv"
    true_path = f"G:\\내 드라이브\\DIMA\\프로젝트\\생활인구데이터\\LOCAL_PEOPLE_DONG_{yyyymm}.csv"

    if not os.path.exists(pred_path) or not os.path.exists(true_path):
        print(f" - 파일 누락: {pred_path if not os.path.exists(pred_path) else true_path}")
        continue

    # 파일 로딩
    pred_df = pd.read_csv(pred_path, encoding='utf-8')
    true_df = pd.read_csv(true_path, encoding='utf-8', index_col=False)

    # 정제
    pred_df['행정동코드'] = pred_df['행정동코드'].astype(str)
    true_df['행정동코드'] = true_df['행정동코드'].astype(float).astype(int).astype(str)
    true_df['colname'] = true_df.apply(lambda r: f"{r['기준일ID']}_{int(r['시간대구분']):02d}", axis=1)

    # 시간 필터
    time_columns = [col for col in pred_df.columns if col.startswith("2023") and len(col) == 11]

    # 집계 및 병합
    pred_by_dong = pred_df.groupby("행정동코드")[time_columns].sum().reset_index()
    true_pivot = true_df.pivot_table(index="행정동코드", columns="colname", values="총생활인구수", aggfunc="sum").reset_index()
    merged = pd.merge(pred_by_dong, true_pivot, on="행정동코드", suffixes=("_예측", "_실제"))

    # 비교 지표 계산
    metrics = []
    for col in time_columns:
        col_pred = f"{col}_예측"
        col_true = f"{col}_실제"
        if col_pred not in merged.columns or col_true not in merged.columns:
            continue

        pred = merged[col_pred]
        true = merged[col_true]
        diff = (pred - true).abs()
        mae = diff.mean()
        rmse = np.sqrt(((pred - true) ** 2).mean())
        total_diff = (pred.sum() - true.sum()) / true.sum() * 100

        metrics.append({
            "시간대": col,
            "MAE": mae,
            "RMSE": rmse,
            "총량오차(%)": total_diff
        })

    month_result_df = pd.DataFrame(metrics)
    print(month_result_df.head(24))

### 2024년 모두 저장하기

In [9]:
# 기본정보
keyword_list = ['주거지역', '상업지역', '공업지역', '전용주거지역', '준공업지역', '근린상업지역', '복합지역']
poi_df = pd.read_csv("./merge/서울시_관광특구_POI_좌표.csv", encoding='cp949')
grid_df = pd.read_csv("./서울시격자_아동인구매핑완료.csv", encoding='utf-8')
grid_df['행정동코드'] = grid_df['행정동코드'].astype(str)
poi_coords = list(zip(poi_df['latitude'], poi_df['longitude']))

# 공통함수
def contains_residential(info):
    try:
        d = ast.literal_eval(info)
        return any(any(k in key for k in keyword_list) for key in d)
    except:
        return False

def calc_distance_weight(lat, lon):
    grid_point = (lat, lon)
    return sum(1 / max(haversine(grid_point, poi), 0.001) for poi in poi_coords)

def calc_centroid(geom):
    try:
        poly = wkt.loads(geom)
        return poly.centroid.y, poly.centroid.x
    except:
        return None, None

def extract_centroid_coords(geom_str):
    try:
        polygon = wkt.loads(geom_str)
        centroid = polygon.centroid
        return pd.Series({'lon': centroid.x, 'lat': centroid.y})
    except:
        return pd.Series({'lon': None, 'lat': None})

# 1년 루프
for yyyymm in [f"2024{m:02d}" for m in range(1, 13)]:
    print(f"처리 중: {yyyymm}")

    path = f"G:\\내 드라이브\\DIMA\\프로젝트\\생활인구데이터\\LOCAL_PEOPLE_DONG_{yyyymm}.csv"
    if not os.path.exists(path):
        print(f"{path} 파일 없음, 건너뜀")
        continue

    # 생활인구 로딩 및 전처리
    people_df = pd.read_csv(path, index_col=False, encoding='utf-8')
    people_df['행정동코드'] = people_df['행정동코드'].astype(float).astype(int).astype(str)

    # 보간 매핑 정의
    보간매핑 = {
        '11305600': ['11305615', '11305625', '11305635'],
        '11305590': ['11305595'],
        '11740520': ['11740525', '11740526']
    }
    전체매핑 = {**보간매핑, **{c: [c] for c in people_df['행정동코드'].unique() if c not in 보간매핑}}

    # 거주 가능 격자 필터링 + 가중치 계산
    res_grids = grid_df[grid_df['지적편집도_상세정보'].apply(contains_residential)].copy()
    res_grids[['lat', 'lon']] = res_grids['geometry'].apply(lambda g: pd.Series(calc_centroid(g)))
    res_grids['거주가중치'] = 1.0
    res_grids['거리가중치'] = res_grids.apply(lambda row: calc_distance_weight(row['lat'], row['lon']), axis=1)
    res_grids['최종가중치'] = res_grids['거주가중치'] * res_grids['거리가중치']

    # 인구 분배
    grid_values = defaultdict(lambda: defaultdict(float))
    for _, row in tqdm(people_df.iterrows(), total=len(people_df)):
        date_id = str(row['기준일ID'])
        hour = int(row['시간대구분'])
        col_name = f"{date_id}_{hour:02d}"
        hd_code = row['행정동코드']
        pop = float(row['총생활인구수'])

        target_codes = 전체매핑.get(hd_code, [hd_code])
        sub_grids = res_grids[res_grids['행정동코드'].isin(target_codes)]

        if sub_grids.empty:
            continue

        weights = sub_grids['최종가중치'].values
        weight_sum = weights.sum()
        if weight_sum == 0:
            continue

        ratios = weights / weight_sum
        for grid_id, ratio in zip(sub_grids['grid_id'], ratios):
            grid_values[grid_id][col_name] += pop * ratio

    # 5. wide 포맷 저장 (모든 격자 유지)
    all_columns = sorted({col for d in grid_values.values() for col in d})
    wide_df = pd.DataFrame(index=grid_df['grid_id'], columns=all_columns, dtype=float).fillna(0.0)

    for grid_id in tqdm(grid_values, desc=f"{yyyymm} wide 생성"):
        for col, value in grid_values[grid_id].items():
            wide_df.at[grid_id, col] = float(value)

    wide_df.reset_index(inplace=True)
    wide_df.rename(columns={'index': 'grid_id'}, inplace=True)

    # 격자와 병합 + 저장 (left join)
    final_df = pd.merge(grid_df, wide_df, on="grid_id", how="left").fillna(0)

    # 최종 저장 전 보간 처리 코드
    interpolation_target_dongs = {
        '항동': ['오류2동', '수궁동'],
        '개포3동': ['잠실본동', '대치2동', '개포2동', '일원본동', '일원1동', '삼전동'],
        '번2동': ['창3동', '미아동', '번1동'],
        '번3동': ['장위1동', '장위3동', '월계2동', '송중동'],
        '상일2동': ['강일동', '고덕2동'],
        '상일1동': ['고덕2동', '고덕1동', '명일2동'],
        '제기동': ['청량리동', '용신동'],
        '홍제1동': ['홍제2동', '홍제3동'],
        '홍제2동': ['홍제1동', '홍제3동'],
        '홍제3동': ['홍제1동', '홍제2동']
    }
    
    # 생활인구 컬럼 정의 (날짜_시간대 형식)
    pop_columns = [col for col in final_df.columns if re.match(r'20\d{6}_\d{2}', col)]
    
    # 거주 가능하지만 생활인구가 0인 격자 확인
    final_df['총생활인구수'] = final_df[pop_columns].sum(axis=1)
    no_pop_grids = final_df[
        (final_df['총생활인구수'] == 0) &
        final_df['지적편집도_상세정보'].apply(contains_residential)
    ]
    
    print(f"보간이 필요한 격자 수: {len(no_pop_grids)}")
    
    # 보간 처리 수행
    for idx, row in tqdm(no_pop_grids.iterrows(), total=len(no_pop_grids), desc="생활인구 보간"):
        target_dong = row['행정동명']
        ref_dongs = interpolation_target_dongs.get(target_dong, [])
        
        if not ref_dongs:
            continue
        
        ref_grids = final_df[final_df['행정동명'].isin(ref_dongs) & (final_df['총생활인구수'] > 0)]
        
        if ref_grids.empty:
            continue
        
        # 참조 동의 평균 생활인구를 구해 보간
        mean_pop = ref_grids[pop_columns].mean()
        final_df.loc[idx, pop_columns] = mean_pop.values
        final_df.loc[idx, '총생활인구수'] = mean_pop.sum()

    # 최종 저장
    final_df.drop(columns=['총생활인구수'], inplace=True)
    final_df.to_csv(f"./서울시격자_{yyyymm}_보간완료.csv", index=False)
    print(f"{yyyymm} 보간완료 저장 완료!")

처리 중: 202401


202401 wide 생성: 100%|████████████████████████████████████████████████████████████| 2034/2034 [00:26<00:00, 75.52it/s]


보간이 필요한 격자 수: 20


생활인구 보간: 100%|███████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  8.17it/s]


202401 보간완료 저장 완료!
처리 중: 202402


...

202411 보간완료 저장 완료!
처리 중: 202412


202412 wide 생성: 100%|████████████████████████████████████████████████████████████| 2034/2034 [00:26<00:00, 76.55it/s]


보간이 필요한 격자 수: 20


생활인구 보간: 100%|███████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  8.97it/s]


202412 보간완료 저장 완료!


### 누락확인

In [10]:
# 키워드와 함수 정의
keyword_list = ['주거지역', '상업지역', '공업지역', '전용주거지역', '준공업지역', '근린상업지역', '복합지역']
grid_df = pd.read_csv("./서울시격자_아동인구매핑완료.csv", encoding='utf-8')
grid_df['행정동코드'] = grid_df['행정동코드'].astype(str)

def contains_residential(info):
    try:
        d = ast.literal_eval(info)
        return any(any(k in key for k in keyword_list) for key in d)
    except:
        return False

# 거주 가능 격자 ID 추출
res_grids = grid_df[grid_df['지적편집도_상세정보'].apply(contains_residential)]
residential_ids = set(res_grids['grid_id'])

# 루프 돌면서 누락 확인
for yyyymm in [f"2024{m:02d}" for m in range(1, 13)]:
    print(f"\n{yyyymm} 검사 중")

    merged_path = f"서울시격자_{yyyymm}_보간완료.csv"
    if not os.path.exists(merged_path):
        print(f" - 파일 없음: {merged_path}")
        continue

    merged_df = pd.read_csv(merged_path)

    # 인구 관련 열 추출
    pop_cols = [col for col in merged_df.columns if re.fullmatch(r'20\d{6}_\d{2}', col)]
    merged_df['인구합'] = merged_df[pop_cols].sum(axis=1)

    # 인구합이 0인 격자
    zero_pop_grids = merged_df[merged_df['인구합'] == 0]['grid_id'].tolist()

    # 거주 가능한데 인구가 0인 격자만 필터링
    missed = [gid for gid in zero_pop_grids if gid in residential_ids]

    print(f" - 거주 가능 격자 중 인구 0인 격자 수: {len(missed)}")
    if missed:
        print(f"   예시: {missed[:5]}")


202401 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202402 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202403 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202404 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202405 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202406 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202407 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202408 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202409 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202410 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202411 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0

202412 검사 중
 - 거주 가능 격자 중 인구 0인 격자 수: 0


### 매핑수치 확인

In [ ]:
for yyyymm in [f"2024{m:02d}" for m in range(1, 13)]:
    print(f"\n{yyyymm} 비교 중")

    pred_path = f"./서울시격자_{yyyymm}_보간완료.csv"
    true_path = f"G:\\내 드라이브\\DIMA\\프로젝트\\생활인구데이터\\LOCAL_PEOPLE_DONG_{yyyymm}.csv"

    if not os.path.exists(pred_path) or not os.path.exists(true_path):
        print(f" - 파일 누락: {pred_path if not os.path.exists(pred_path) else true_path}")
        continue

    # 파일 로딩
    pred_df = pd.read_csv(pred_path, encoding='utf-8')
    true_df = pd.read_csv(true_path, encoding='utf-8', index_col=False)

    # 정제
    pred_df['행정동코드'] = pred_df['행정동코드'].astype(str)
    true_df['행정동코드'] = true_df['행정동코드'].astype(float).astype(int).astype(str)
    true_df['colname'] = true_df.apply(lambda r: f"{r['기준일ID']}_{int(r['시간대구분']):02d}", axis=1)

    # 시간 필터
    time_columns = [col for col in pred_df.columns if col.startswith("2024") and len(col) == 11]

    # 집계 및 병합
    pred_by_dong = pred_df.groupby("행정동코드")[time_columns].sum().reset_index()
    true_pivot = true_df.pivot_table(index="행정동코드", columns="colname", values="총생활인구수", aggfunc="sum").reset_index()
    merged = pd.merge(pred_by_dong, true_pivot, on="행정동코드", suffixes=("_예측", "_실제"))

    # 비교 지표 계산
    metrics = []
    for col in time_columns:
        col_pred = f"{col}_예측"
        col_true = f"{col}_실제"
        if col_pred not in merged.columns or col_true not in merged.columns:
            continue

        pred = merged[col_pred]
        true = merged[col_true]
        diff = (pred - true).abs()
        mae = diff.mean()
        rmse = np.sqrt(((pred - true) ** 2).mean())
        total_diff = (pred.sum() - true.sum()) / true.sum() * 100

        metrics.append({
            "시간대": col,
            "MAE": mae,
            "RMSE": rmse,
            "총량오차(%)": total_diff
        })

    month_result_df = pd.DataFrame(metrics)
    print(month_result_df.head(24))

### 케플러 파일 조정 저장

In [ ]:
import pandas as pd
import shapely.wkt
from shapely.geometry import Point
from datetime import datetime
from tqdm import tqdm

tqdm.pandas()

# CSV 로드
df = pd.read_csv("./서울시격자_202412_보간완료.csv")
df["geometry_obj"] = df["geometry"].progress_apply(shapely.wkt.loads)

# 아동인구 CSV (Polygon)
print("아동인구 CSV 저장 시작")
df["아동인구"] = df["2024_0-4세"] + df["2024_5-9세"]
df_out = df[["grid_id", "아동인구", "geometry"]].copy()
df_out.to_csv("./시각화용파일/kepler_아동인구.csv", index=False, encoding="utf-8-sig")
print("kepler_아동인구.csv 저장 완료")


# 생활인구 CSV (Polygon + 시간별)
print("생활인구 시간별 CSV 저장 시작")
pop_cols = [col for col in df.columns if col.startswith("202412")]

rows = []
for i in tqdm(range(len(df)), desc="생활인구"):
    row = df.iloc[i]
    for col in pop_cols:
        try:
            dt = datetime.strptime(col, "%Y%m%d_%H")
            value = row[col]
            if pd.notnull(value):
                rows.append({
                    "grid_id": row["grid_id"],
                    "datetime": dt.strftime("%Y-%m-%d %H:%M:%S"),
                    "생활인구": int(value),
                    "geometry": row["geometry"]
                })
        except:
            continue

df_pop = pd.DataFrame(rows)
df_pop.to_csv("./시각화용파일/kepler_생활인구_시간별.csv", index=False, encoding="utf-8-sig")
print("kepler_생활인구_시간별.csv 저장 완료")


# 시설정보 CSV (Point만)
print("시설정보 CSV 저장 시작")
facility_cols = [
    "서울형공립키즈카페_정보",
    "인증제키즈카페_정보",
    "일반키즈카페_정보",
    "유치원_초등학교_정보",
    "어린이집_정보",
    "지역아동센터_정보",
    "키움센터_정보",
    "유흥시설_정보",
    "대중교통_정보",
]

records = []
for col in tqdm(facility_cols, desc="시설 처리"):
    category = col.replace("_정보", "")
    for wkt_str in df[col].dropna().unique():
        try:
            pt = shapely.wkt.loads(wkt_str)
            if isinstance(pt, Point):
                records.append({
                    "category": category,
                    "custom_category": category,
                    "lon": round(pt.x, 6),
                    "lat": round(pt.y, 6)
                })
        except:
            continue

df_facility = pd.DataFrame(records)
df_facility.to_csv("./시각화용파일/kepler_시설정보_포인트.csv", index=False, encoding="utf-8-sig")
print("kepler_시설정보_포인트.csv 저장 완료")

In [ ]:
import pandas as pd
import shapely.wkt
from datetime import datetime
from tqdm import tqdm

# CSV 로드
df = pd.read_csv("./서울시격자_202412.csv")
df["geometry_obj"] = df["geometry"].apply(shapely.wkt.loads)

# 시간대 컬럼만 추출
pop_cols = [col for col in df.columns if col.startswith("202412")]

# long 형식 stream 저장
print("생활인구 long 형식 stream 저장 시작")
with open("./시각화용파일/kepler_생활인구_시간별.csv", "w", encoding="utf-8-sig") as f:
    f.write("grid_id,datetime,생활인구,geometry\n")  # 헤더

    for i in tqdm(range(len(df)), desc="Row 저장"):
        row = df.iloc[i]
        for col in pop_cols:
            try:
                dt = datetime.strptime(col, "%Y%m%d_%H")
                val = row[col]
                if pd.notnull(val):
                    line = f"{row['grid_id']},{dt.strftime('%Y-%m-%d %H:%M:%S')},{int(val)},{row['geometry']}\n"
                    f.write(line)
            except:
                continue

print("kepler_생활인구_시간별.csv 저장 완료 (long format)")

In [ ]:
import pandas as pd
import shapely.wkt
from datetime import datetime
from tqdm import tqdm

# 원본 CSV 로드
df = pd.read_csv("./서울시격자_202412.csv")
df["geometry_obj"] = df["geometry"].apply(shapely.wkt.loads)

# 하반기 대상 컬럼만 추출 (2024년 7~12월)
target_months = ["202407", "202408", "202409", "202410", "202411", "202412"]
pop_cols = [col for col in df.columns if any(col.startswith(month) for month in target_months)]

# stream 방식으로 long format 저장
print("2024 하반기 생활인구 long 형식 stream 저장 시작")
with open("./시각화용파일/kepler_생활인구_2024하반기.csv", "w", encoding="utf-8-sig") as f:
    f.write("grid_id,datetime,생활인구,geometry\n")  # 헤더

    for i in tqdm(range(len(df)), desc="Row 저장"):
        row = df.iloc[i]
        for col in pop_cols:
            try:
                dt = datetime.strptime(col, "%Y%m%d_%H")
                val = row[col]
                if pd.notnull(val):
                    line = f"{row['grid_id']},{dt.strftime('%Y-%m-%d %H:%M:%S')},{int(val)},{row['geometry']}\n"
                    f.write(line)
            except:
                continue

print("kepler_생활인구_2024하반기.csv 저장 완료")

In [ ]:
import pandas as pd
import shapely.wkt
from datetime import datetime
from tqdm import tqdm

# 원본 CSV 로드
df = pd.read_csv("./서울시격자_202412.csv")
df["geometry_obj"] = df["geometry"].apply(shapely.wkt.loads)

# 2024년 4분기 컬럼만 추출
target_months = ["202410", "202411", "202412"]
pop_cols = [col for col in df.columns if any(col.startswith(month) for month in target_months)]

# stream 방식으로 long format 저장
print("2024 4분기 생활인구 stream 저장 시작")
with open("./시각화용파일/kepler_생활인구_2024Q4.csv", "w", encoding="utf-8-sig") as f:
    f.write("grid_id,datetime,생활인구,geometry\n")  # 헤더

    for i in tqdm(range(len(df)), desc="Row 저장"):
        row = df.iloc[i]
        for col in pop_cols:
            try:
                dt = datetime.strptime(col, "%Y%m%d_%H")
                val = row[col]
                if pd.notnull(val):
                    line = f"{row['grid_id']},{dt.strftime('%Y-%m-%d %H:%M:%S')},{int(val)},{row['geometry']}\n"
                    f.write(line)
            except:
                continue

print("kepler_생활인구_2024Q4.csv 저장 완료")

In [ ]:
import pandas as pd
import shapely.wkt
from datetime import datetime
from tqdm import tqdm

# CSV 로드
df = pd.read_csv("./서울시격자_202412.csv")
df["geometry_obj"] = df["geometry"].apply(shapely.wkt.loads)

# 202412만 필터링
pop_cols = [col for col in df.columns if col.startswith("202412")]

# stream 저장
print("2024년 12월 생활인구 저장 시작")
with open("./시각화용파일/kepler_생활인구_202412.csv", "w", encoding="utf-8-sig") as f:
    f.write("grid_id,datetime,생활인구,geometry\n")  # 헤더

    for i in tqdm(range(len(df)), desc="Row 저장"):
        row = df.iloc[i]
        for col in pop_cols:
            try:
                dt = datetime.strptime(col, "%Y%m%d_%H")
                val = row[col]
                if pd.notnull(val):
                    f.write(f"{row['grid_id']},{dt.strftime('%Y-%m-%d %H:%M:%S')},{int(val)},{row['geometry']}\n")
            except:
                continue

print("kepler_생활인구_202412.csv 저장 완료")

In [ ]:
import pandas as pd
import ast  # 문자열 리스트 safely eval
from tqdm import tqdm

df = pd.read_csv("./서울시격자_202412_보간완료.csv")

facility_cols = [
    "서울형공립키즈카페_정보",
    "인증제키즈카페_정보",
    "일반키즈카페_정보",
    "유치원_초등학교_정보",
    "어린이집_정보",
    "지역아동센터_정보",
    "키움센터_정보",
    "유흥시설_정보",
    "대중교통_정보",
]

records = []

for col in tqdm(facility_cols, desc="시설 포인트 추출"):
    category = col.replace("_정보", "")
    for raw in df[col].dropna().unique():
        try:
            facility_list = ast.literal_eval(raw)
            for item in facility_list:
                lat = float(item.get("위도", None))
                lon = float(item.get("경도", None))
                if lat and lon:
                    records.append({
                        "category": category,
                        "custom_category": category,
                        "lat": round(lat, 6),
                        "lon": round(lon, 6)
                    })
        except Exception as e:
            continue  # malformed JSON-like string, just skip

df_facility = pd.DataFrame(records)
df_facility.to_csv("./시각화용파일/kepler_시설정보_포인트.csv", index=False, encoding="utf-8-sig")
print(f"저장 완료! 시설 수: {len(df_facility)}개")

In [ ]:
import pandas as pd
import shapely.wkt
from datetime import datetime
from tqdm import tqdm

# 원본 CSV 로드
df = pd.read_csv("./서울시격자_202412_보간완료.csv")

# geometry 파싱
df["geometry_obj"] = df["geometry"].apply(shapely.wkt.loads)

# 2024-12-01 ~ 2024-12-03 기간만 추출
target_days = [f"202412{str(day).zfill(2)}" for day in range(1, 4)]
pop_cols = [col for col in df.columns if any(col.startswith(day) for day in target_days)]

# stream 방식 long format 저장
print("2024년 12월 1~3일 생활인구 저장 시작...")
with open("./시각화용파일/kepler_생활인구_202412_3일치.csv", "w", encoding="utf-8-sig") as f:
    f.write("grid_id,datetime,생활인구,geometry\n")

    for i in tqdm(range(len(df)), desc="Row 저장"):
        row = df.iloc[i]
        for col in pop_cols:
            try:
                dt = datetime.strptime(col, "%Y%m%d_%H")
                val = row[col]
                if pd.notnull(val):
                    wkt_geom = f'"{row["geometry"]}"'  # WKT 따옴표 처리
                    f.write(f"{row['grid_id']},{dt.strftime('%Y-%m-%d %H:%M:%S')},{int(val)},{wkt_geom}\n")
            except:
                continue

print("kepler_생활인구_202412_3일치.csv 저장 완료")